# Лабораторна робота №4
**Мета:** Ознайомитися з роботою над великими обсягами даних у Python за допомогою `pandas DataFrame` та `NumPy array`, порівняти продуктивність та зручність відбору даних.

**Задачі:**
1. Фільтрація записів, де `Global_active_power > 5.0`.
2. Фільтрація записів, де `Voltage > 235`.
3. Фільтрація записів, де `Global_intensity` знаходиться в межах 19-20, а `Sub_metering_2` перевищує `Sub_metering_1` та `Sub_metering_3`.
4. Випадкова вибірка 500000 записів та обчислення середніх значень для груп споживання.
5. Складна фільтрація записів за умовами часу, потужності та домінування споживання, з подальшим розбиттям вибірки.


In [35]:
import time
import numpy as np
import pandas as pd
from datetime import time as dt_time
from tabulate import tabulate

print("Налаштування завершено")

Налаштування завершено


In [36]:
data_file = "household_power_consumption.txt"

# Функція для відображення перших кількох рядків даних
def show_head(data, n=20):
    if isinstance(data, pd.DataFrame):
        print("Перші рядки DataFrame:")
        print(tabulate(data.head(n), headers='keys', tablefmt='psql'))
    elif isinstance(data, np.ndarray):
        print("Перші рядки NumPy масиву:")
        print(tabulate(data[:n], tablefmt='psql'))
    else:
        print("Невідомий тип даних.")

# Функція завантаження даних у DataFrame
def load_df(filename):
    columns = ["Date", "Time", "Global_active_power", "Global_reactive_power", 
               "Voltage", "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]
    dtype_spec = {"Global_active_power": float, "Global_reactive_power": float,
                  "Voltage": float, "Global_intensity": float,
                  "Sub_metering_1": float, "Sub_metering_2": float, "Sub_metering_3": float}
    
    # Перевірте, чи правильний роздільник у вашому файлі (тут використовується ";" як роздільник)
    df = pd.read_csv(filename, sep=";", header=0, names=columns, dtype=dtype_spec, na_values=["?"], low_memory=False)
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    df["Time"] = pd.to_datetime(df["Time"], format="%H:%M:%S", errors='coerce').dt.time
    df = df.dropna()
    return df

# Функція завантаження даних як NumPy масиву через DataFrame
def load_np(filename):
    df = load_df(filename)
    return df.to_numpy()

# Завантаження даних
df = load_df(data_file)
np_data = load_np(data_file)

print("Розміри даних (рядки, колонки):", df.shape)
show_head(df)

Розміри даних (рядки, колонки): (2049280, 9)
Перші рядки DataFrame:
+----+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------+
|    | Date                | Time     |   Global_active_power |   Global_reactive_power |   Voltage |   Global_intensity |   Sub_metering_1 |   Sub_metering_2 |   Sub_metering_3 |
|----+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------|
|  0 | 2006-12-16 00:00:00 | 17:24:00 |                 4.216 |                   0.418 |    234.84 |               18.4 |                0 |                1 |               17 |
|  1 | 2006-12-16 00:00:00 | 17:25:00 |                 5.36  |                   0.436 |    233.63 |               23   |                0 |                1 |               16 |
|  2 | 2006-12-16 00:00:00 | 17:

Задача 1 – Фільтрація за Global_active_power > 5.0

In [37]:
# Фільтрація для DataFrame
def filter_power_pd(df):
    return df[df["Global_active_power"] > 5.0]

# Фільтрація для NumPy масиву (Global_active_power знаходиться за індексом 2)
def filter_power_np(arr):
    return arr[arr[:, 2] > 5.0]

start = time.perf_counter()
filtered_pd_1 = filter_power_pd(df)
time_pd_1 = time.perf_counter() - start

start = time.perf_counter()
filtered_np_1 = filter_power_np(np_data)
time_np_1 = time.perf_counter() - start

print("\nЗадача 1: Фільтрація записів за умовою Global_active_power > 5.0")
show_head(filtered_pd_1)
show_head(filtered_np_1)
print(f"Час виконання: DataFrame: {time_pd_1:.6f} сек, NumPy: {time_np_1:.6f} сек")



Задача 1: Фільтрація записів за умовою Global_active_power > 5.0
Перші рядки DataFrame:
+-----+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------+
|     | Date                | Time     |   Global_active_power |   Global_reactive_power |   Voltage |   Global_intensity |   Sub_metering_1 |   Sub_metering_2 |   Sub_metering_3 |
|-----+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------|
|   1 | 2006-12-16 00:00:00 | 17:25:00 |                 5.36  |                   0.436 |    233.63 |               23   |                0 |                1 |               16 |
|   2 | 2006-12-16 00:00:00 | 17:26:00 |                 5.374 |                   0.498 |    233.29 |               23   |                0 |                2 |               17 |
|   3 

Задача 2 – Фільтрація за Voltage > 235

In [38]:
# Фільтрація для DataFrame
def filter_voltage_pd(df):
    return df[df["Voltage"] > 235]

# Фільтрація для NumPy масиву (Voltage знаходиться за індексом 4)
def filter_voltage_np(arr):
    return arr[arr[:, 4] > 235]

start = time.perf_counter()
filtered_pd_2 = filter_voltage_pd(df)
time_pd_2 = time.perf_counter() - start

start = time.perf_counter()
filtered_np_2 = filter_voltage_np(np_data)
time_np_2 = time.perf_counter() - start

print("\nЗадача 2: Фільтрація записів за умовою Voltage > 235")
show_head(filtered_pd_2)
show_head(filtered_np_2)
print(f"Час виконання: DataFrame: {time_pd_2:.6f} сек, NumPy: {time_np_2:.6f} сек")



Задача 2: Фільтрація записів за умовою Voltage > 235
Перші рядки DataFrame:
+----+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------+
|    | Date                | Time     |   Global_active_power |   Global_reactive_power |   Voltage |   Global_intensity |   Sub_metering_1 |   Sub_metering_2 |   Sub_metering_3 |
|----+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------|
|  4 | 2006-12-16 00:00:00 | 17:28:00 |                 3.666 |                   0.528 |    235.68 |               15.8 |                0 |                1 |               17 |
|  5 | 2006-12-16 00:00:00 | 17:29:00 |                 3.52  |                   0.522 |    235.02 |               15   |                0 |                2 |               17 |
|  6 | 2006-12-16 00:00

 Задача 3 – Фільтрація за Global_intensity (19-20) та домінуванням Sub_metering_2

In [39]:
# Фільтрація для DataFrame
def filter_current_consumption_pd(df):
    cond_current = (df["Global_intensity"] >= 19) & (df["Global_intensity"] <= 20)
    cond_consumption = (df["Sub_metering_2"] > df["Sub_metering_1"]) & (df["Sub_metering_2"] > df["Sub_metering_3"])
    return df[cond_current & cond_consumption]

# Фільтрація для NumPy масиву (Global_intensity – індекс 5, Sub_metering_1,2,3 – індекси 6,7,8)
def filter_current_consumption_np(arr):
    cond_current = (arr[:, 5] >= 19) & (arr[:, 5] <= 20)
    cond_consumption = (arr[:, 7] > arr[:, 6]) & (arr[:, 7] > arr[:, 8])
    return arr[cond_current & cond_consumption]

start = time.perf_counter()
filtered_pd_3 = filter_current_consumption_pd(df)
time_pd_3 = time.perf_counter() - start

start = time.perf_counter()
filtered_np_3 = filter_current_consumption_np(np_data)
time_np_3 = time.perf_counter() - start

print("\nЗадача 3: Фільтрація за Global_intensity (19-20) та домінуванням Sub_metering_2")
show_head(filtered_pd_3)
show_head(filtered_np_3)
print(f"Час виконання: DataFrame: {time_pd_3:.6f} сек, NumPy: {time_np_3:.6f} сек")



Задача 3: Фільтрація за Global_intensity (19-20) та домінуванням Sub_metering_2
Перші рядки DataFrame:
+------+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------+
|      | Date                | Time     |   Global_active_power |   Global_reactive_power |   Voltage |   Global_intensity |   Sub_metering_1 |   Sub_metering_2 |   Sub_metering_3 |
|------+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------|
|   45 | 2006-12-16 00:00:00 | 18:09:00 |                 4.464 |                   0.136 |    234.66 |               19   |                0 |               37 |               16 |
|  460 | 2006-12-17 00:00:00 | 01:04:00 |                 4.582 |                   0.258 |    238.08 |               19.6 |                0 |               13 |      

Задача 4 – Випадкова вибірка 500000 записів та обчислення середніх значень

In [40]:
def random_sample_avg_pd(df, n=500000):
    replace_flag = n > len(df)
    sample_df = df.sample(n=n, replace=replace_flag, random_state=42)
    return sample_df[["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]].mean()

# Функція для NumPy масиву: використання np.random.choice
def random_sample_avg_np(arr, n=500000):
    replace_flag = n > arr.shape[0]
    indices = np.random.choice(arr.shape[0], n, replace=replace_flag)
    sample_arr = arr[indices, 6:9]
    return np.mean(sample_arr, axis=0)

start = time.perf_counter()
avg_pd = random_sample_avg_pd(df)
time_pd_4 = time.perf_counter() - start

start = time.perf_counter()
avg_np = random_sample_avg_np(np_data)
time_np_4 = time.perf_counter() - start

print("\nЗадача 4: Випадкова вибірка 500000 записів та обчислення середніх значень споживання")
print("Середні значення (DataFrame):")
print(avg_pd)
print("Середні значення (NumPy):")
print(avg_np)
print(f"Час виконання: DataFrame: {time_pd_4:.6f} сек, NumPy: {time_np_4:.6f} сек")


Задача 4: Випадкова вибірка 500000 записів та обчислення середніх значень споживання
Середні значення (DataFrame):
Sub_metering_1    1.119258
Sub_metering_2    1.308912
Sub_metering_3    6.452950
dtype: float64
Середні значення (NumPy):
[1.13171 1.299076 6.459878]
Час виконання: DataFrame: 0.558692 сек, NumPy: 0.725615 сек


Задача 5 – Складна фільтрація та подальший відбір



In [41]:
# Фільтрація для DataFrame
def complex_filter_pd(df):
    cond_time = df["Time"] > dt_time(18, 0, 0)
    cond_power = df["Global_active_power"] > 6
    cond_consumption = (df["Sub_metering_2"] > df["Sub_metering_1"]) & (df["Sub_metering_2"] > df["Sub_metering_3"])
    filtered = df[cond_time & cond_power & cond_consumption]
    half_index = len(filtered) // 2
    first_half = filtered.iloc[:half_index]
    second_half = filtered.iloc[half_index:]
    sel_first = first_half.iloc[2::3]  # кожен третій запис з першої половини
    sel_second = second_half.iloc[3::4]  # кожен четвертий запис з другої половини
    return pd.concat([sel_first, sel_second])

# Фільтрація для NumPy масиву
def complex_filter_np(arr):
    # Конвертуємо час з рядка у об'єкт time
    times = np.array([pd.to_datetime(t, format="%H:%M:%S").time() for t in arr[:, 1]])
    cond_time = np.array([t > dt_time(18, 0, 0) for t in times])
    cond_power = arr[:, 2] > 6
    cond_consumption = (arr[:, 7] > arr[:, 6]) & (arr[:, 7] > arr[:, 8])
    overall = cond_time & cond_power & cond_consumption
    filtered = arr[overall]
    half = len(filtered) // 2
    first_half = filtered[:half]
    second_half = filtered[half:]
    sel_first = first_half[2::3]
    sel_second = second_half[3::4]
    return np.concatenate([sel_first, sel_second], axis=0)

start = time.perf_counter()
complex_pd = complex_filter_pd(df)
time_pd_5 = time.perf_counter() - start

start = time.perf_counter()
complex_np = complex_filter_np(np_data)
time_np_5 = time.perf_counter() - start

print("\nЗадача 5: Складна фільтрація та подальший відбір записів")
show_head(complex_pd)
show_head(complex_np)
print(f"Час виконання: DataFrame: {time_pd_5:.6f} сек, NumPy: {time_np_5:.6f} сек")



Задача 5: Складна фільтрація та подальший відбір записів
Перші рядки DataFrame:
+-------+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------+
|       | Date                | Time     |   Global_active_power |   Global_reactive_power |   Voltage |   Global_intensity |   Sub_metering_1 |   Sub_metering_2 |   Sub_metering_3 |
|-------+---------------------+----------+-----------------------+-------------------------+-----------+--------------------+------------------+------------------+------------------|
|    43 | 2006-12-16 00:00:00 | 18:07:00 |                 6.474 |                   0.144 |    231.85 |               27.8 |                0 |               37 |               16 |
|  3007 | 2006-12-18 00:00:00 | 19:31:00 |                 6.158 |                   0.442 |    229.08 |               27   |                0 |               36 |                0 |
| 17